In [19]:
from PIL import Image, ImageOps
import os
import pandas as pd
import glob
import numpy as np
import shutil
from tqdm.auto import tqdm
from utility import check_yolo_label
import uuid
import random
import cv2

DRONS_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\Phantom 4 CGI 3"
BACKGROUND_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\background_from_antiUAV"
GENERATE_DIR = r"C:\project\Computer_Vision\term_paper_2023\generate_Phantom4_CGI_background_antiUAV_3"
# DRONS_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\dataset_drone_without_background_crop"
# BACKGROUND_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\background_for_generate"
# GENERATE_DIR = r"C:\project\Computer_Vision\term_paper_2023\generate_drone_1024"
IMAGES_DIR = os.path.join(GENERATE_DIR, "images")
MASKS_DIR = os.path.join(GENERATE_DIR, 'masks')
YOLO_LABELS_DIR = os.path.join(GENERATE_DIR, 'labels')
CHECK_YOLO_LABELS_DIR = os.path.join(GENERATE_DIR, 'check_labels')

In [20]:
if os.path.exists(YOLO_LABELS_DIR): shutil.rmtree(YOLO_LABELS_DIR)
if os.path.exists(IMAGES_DIR): shutil.rmtree(IMAGES_DIR)
if os.path.exists(MASKS_DIR): shutil.rmtree(MASKS_DIR)
if os.path.exists(CHECK_YOLO_LABELS_DIR): shutil.rmtree(CHECK_YOLO_LABELS_DIR)
os.makedirs(YOLO_LABELS_DIR)
os.makedirs(IMAGES_DIR)
os.makedirs(MASKS_DIR)
os.makedirs(CHECK_YOLO_LABELS_DIR)

In [21]:
def get_dron_df():
    df = pd.DataFrame (glob.glob( os.path.join(DRONS_DIR,'*.png')), columns=['path_image'])
    df['id'] = df['path_image'].apply(lambda x : x.split('\\')[-1].split('.')[0])
    return df

def get_background_df():
    df = pd.DataFrame (glob.glob( os.path.join(BACKGROUND_DIR,'*.jpg') ), columns=['path_image'])
    df['id'] = df['path_image'].apply(lambda x : x.split('\\')[-1].split('.')[0])
    return df

df_drons = get_dron_df()
df_background = get_background_df()

In [22]:
df_background

,path_image,id
0,C:\project\Computer_Vision\term_paper_2023\dat...,00344
1,C:\project\Computer_Vision\term_paper_2023\dat...,00363
2,C:\project\Computer_Vision\term_paper_2023\dat...,00366
3,C:\project\Computer_Vision\term_paper_2023\dat...,00380
4,C:\project\Computer_Vision\term_paper_2023\dat...,00381
...,...,...
3817,C:\project\Computer_Vision\term_paper_2023\dat...,05196
3818,C:\project\Computer_Vision\term_paper_2023\dat...,05197
3819,C:\project\Computer_Vision\term_paper_2023\dat...,05198
3820,C:\project\Computer_Vision\term_paper_2023\dat...,05199


In [23]:
# generate_drone = r"C:\project\Computer_Vision\term_paper_2023\experiments\Anti UAV with small drone\generate\images"
# df = pd.DataFrame (glob.glob( os.path.join(generate_drone,'*.jpg')), columns=['path_image'])
# df['id'] = df['path_image'].apply(lambda x : x.split('\\')[-1].split('.')[0])
# df['background'] = df['id'].apply(lambda x : '_'.join(x.split('_')[2:])  )
# df['background_path'] = df['background'].apply(lambda x : os.path.join(BACKGROUND_DIR,f"{x}.jpg")  )
# for i,row in df.iterrows():
#     save_path = r"C:\project\Computer_Vision\term_paper_2023\datasets\background_for_generate"
#     save_path = os.path.join(save_path,f"{row['background']}.jpg")
#     shutil.copyfile(row['background_path'], save_path)

In [24]:
def to_binary(img):
    T = 125
    target = (np.array(img.split()[-1])>T).astype(int)
    target = np.stack((target, target, target), axis=2) * 255
    return Image.fromarray(np.uint8(target)).convert('RGBA')

part_area_drone = []
def compression_drone(drone,drone_mask, image_size, part_percent):
    drone_width, drone_height = drone.size
    ratio = drone_width / drone_height
    area_drone = image_size*image_size*part_percent
    new_height = (area_drone/ratio)**(0.5) * random.uniform(0.8, 1.2)
    new_width = ratio*new_height * random.uniform(0.8, 1.2)
    return drone.resize((int(new_width),int(new_height))), drone_mask.resize((int(new_width),int(new_height)))


def generate_raw(n,min_area, max_area ):
    drons = df_drons.sample(n, replace=True)
    path_drons = drons['path_image'].to_list()
    id_drons = drons['id'].to_list()
    background = df_background.sample(n, replace=True)
    path_backgrounds = background['path_image'].to_list()
    id_backgrounds = background['id'].to_list()
    id_image = []
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    output_size = 1024
    for i in tqdm(range(n)):
        try:
            drone = Image.open(path_drons[i]).convert("RGBA")

            if random.uniform(0, 1) > 0.5:
                drone = ImageOps.mirror(drone)
            # drone = drone_to_red(drone)
            # image = cv2.imread(path_drons[i])
            # img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            # img_hsv[:, :, 1] = 180
            # img = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
            # drone = Image.fromarray(img).convert("RGBA")
            # drone.show()
            # break
            background = Image.open(path_backgrounds[i]).convert("RGB")

            # background_width, background_height = background.size
            #
            # x_crop_background = np.random.randint(0, background_width - output_size, 1)[0]
            # y_crop_background = np.random.randint(0, (background_height - output_size) * 0.3, 1)[0]



            # background = background.crop((x_crop_background,
            #                               y_crop_background,
            #                               x_crop_background + output_size,
            #                               y_crop_background + output_size))

            background = background.resize((output_size,output_size))

            drone,drone_mask = compression_drone(drone,drone, output_size, np.random.uniform(min_area, max_area, 1)[0])

            drone_width, drone_height = drone.size
            # print(drone_width*drone_height)
            x_dron = np.random.randint(0, output_size - drone_width, 1)[0]
            y_dron = np.random.randint(0, output_size - drone_height, 1)[0]

            background.paste(drone, (x_dron, y_dron), drone_mask)

            # SAVE MASK
            mask = Image.new("RGBA", (output_size, output_size), (0, 0, 0, 255))
            bin_drone = to_binary(drone)
            mask.paste(bin_drone, (x_dron, y_dron), drone)
            mask.save(
                os.path.join(MASKS_DIR, f"{str(i)}_{str(id_drons[i])}_{str(id_backgrounds[i])}.png"))

            # mask = Image.new("RGBA", (512, 512), (0, 0, 0, 255))
            # rect = Image.new("RGBA", (dron_width, dron_height), (255, 255, 255, 255))
            # mask.paste(rect, (x_dron, y_dron), rect)
            # mask.save(
            #       os.path.join(MASKS_DIR, f"{str(i)}_{str(id_drons[i])}_{str(id_backgrounds[i])}.png"))

            # SAVE GENERATE IMAGE
            path_image_save = os.path.join(IMAGES_DIR , f"{str(i)}_{str(id_drons[i])}_{str(id_backgrounds[i])}.jpg")
            background.save(path_image_save)

            # FOR CSV
            xmin.append(x_dron)
            ymin.append(y_dron)
            xmax.append(x_dron + drone_width)
            ymax.append(y_dron + drone_height)
            id_image.append(i)

            # YOLO LABELS
            width = drone_width / output_size
            height = drone_height / output_size
            x_center = (x_dron + drone_width / 2) / output_size
            y_center = (y_dron + drone_height / 2) / output_size

            str_ = "0 " + str(x_center) + " " + str(y_center) + " " + str(width) + " " + str(height)

            path_label_coco = os.path.join(YOLO_LABELS_DIR, f"{str(i)}_{str(id_drons[i])}_{str(id_backgrounds[i])}.txt")
            with open(path_label_coco, 'w') as f:
                f.write(str_)

            path_check_yolo_image = os.path.join(CHECK_YOLO_LABELS_DIR , f"{str(i)}_{str(id_drons[i])}_{str(id_backgrounds[i])}.jpg")
            check_yolo_label(Image.open(path_image_save), x_center, y_center, width, height, save_path=path_check_yolo_image )

            part_area_drone.append(width*height)
        except Exception as e:
            print(e)

    df = pd.DataFrame()
    df['xmin'] = xmin
    df['ymin'] = ymin
    df['xmax'] = xmax
    df['ymax'] = ymax
    df['id_image'] = id_image
    df['part_area_drone'] = part_area_drone
    df.to_csv(os.path.join(GENERATE_DIR, "markup.csv"))

#delete trash
# for i,row in tqdm(df_background.iterrows()):
#     img = Image.open(row['path_image'])
#     np_img =  np.asarray( img )
#     if  len(np_img.shape) != 3 or np_img.shape[2] != 3:
#         os.remove(row['path_image'])

# print(df_drons)
generate_raw(1500,0.0005,0.001)

  0%|          | 0/1500 [00:00<?, ?it/s]